In [3]:
import sqlite3

conn = sqlite3.connect('../data/my_letterboxd_data.db')
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT letterboxd_slug FROM film_details_small")
films = cursor.fetchall()
print(films)


[('10-cloverfield-lane',), ('10-rillington-place',), ('10-things-i-hate-about-you',), ('10-to-midnight',), ('10-years',), ('10000-bc',), ('101-dalmatians',), ('101-dalmatians-ii-patchs-london-adventure',), ('102-dalmatians',), ('106013',), ('11-14',), ('12-angry-men',), ('12-angry-men-1997',), ('12-feet-deep',), ('12-hour-shift',), ('12-rounds',), ('12-strong',), ('12-years-a-slave',), ('127-hours',), ('12th-fail',), ('13-assassins',), ('13-cameras',), ('13-ghosts',), ('13-going-on-30',), ('13-hours-the-secret-soldiers-of-benghazi',), ('13-sins',), ('13th',), ('14-peaks-nothing-is-impossible',), ('1492-conquest-of-paradise',), ('15-minutes',), ('16-blocks',), ('16-wishes',), ('17-again',), ('1989-world-tour',), ('1990-the-bronx-warriors',), ('1br',), ('2-days-in-paris',), ('2-days-in-the-valley',), ('2-fast-2-furious',), ('2-guns',), ('2-hearts',), ('2-in-the-am-pm',), ('2-or-3-things-i-know-about-her',), ('2-states',), ('20-days-in-mariupol',), ('20-feet-from-stardom',), ('20-million-

In [13]:
import sqlite3
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import time
from requests.exceptions import ConnectionError, Timeout
import backoff
import json

api_key = "c07c9d067d630faff2eed10b673199a3"

@backoff.on_exception(backoff.expo, (ConnectionError, Timeout), max_tries=25)
def get_film_info(title, year=None):

    # search_response = requests.get(search_url, headers=headers)
    original_title = title
    # if title is in the format film-title-year, remove the year by checking if the last 4 characters are digits
    if title[-4:].isdigit():
        title = title[:-5]

    # if the title is in the format film-title-year-1, remove the year by checking if the last 5 characters are digits
    if title[-1:].isdigit() and title[-6:-2].isdigit():
        title = title[:-7]
        
    search_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}"
    if year:
        search_url += f"&year={year}"
    search_response = requests.get(search_url)
    search_data = search_response.json()
    # print(title)
    if search_data['results']:
        first_result = search_data['results'][0]
        film_id = first_result['id']

        # Fetch additional details (e.g., credits for director and actors)
        details_url = f"https://api.themoviedb.org/3/movie/{film_id}?api_key={api_key}&append_to_response=credits"
        details_response = requests.get(details_url)
        details_data = details_response.json()
        poster_path= details_data.get('poster_path', None)  # Path to the movie poster image



    return poster_path
def main():
    conn = sqlite3.connect('../data/my_letterboxd_data.db')
    cursor = conn.cursor()

    # Get films from the database
    cursor.execute("SELECT DISTINCT letterboxd_slug FROM film_details_small")
    films = cursor.fetchall()

    # Load processed films
   
    processed_films = []

    print(f"Processing {len(films)} films")

    # Process each film
    slug_to_poster = {}
    for film in tqdm(films):
        film_slug = film[0].strip('/')
        if film_slug not in processed_films:
            try:
                poster_path = get_film_info(film_slug)
                if poster_path:
                    poster_path = f"https://image.tmdb.org/t/p/original{poster_path}"
                    slug_to_poster[film_slug] = poster_path
                    print(film_slug, poster_path)
                else:
                    with open('films_not_found.txt', 'a') as f:
                        # Write film slug to file to mark it as not found
                        f.write(f"{film_slug}\n")
            except Exception as e:
                print(f"Error processing film: {film_slug} - {str(e)}")

    conn.close()
    return slug_to_poster

if __name__ == "__main__":
    slug_to_poster = main()
    # save the poster dictionary to a file to be loaded later
    with open('../data/slug_to_poster.json', 'w') as f:
        json.dump(slug_to_poster, f)



Processing 12137 films


  0%|          | 1/12137 [00:00<1:19:52,  2.53it/s]

10-cloverfield-lane https://image.tmdb.org/t/p/original/84Dhwz93vCin6T1PX6ctSvWEuNE.jpg


  0%|          | 3/12137 [00:00<52:11,  3.87it/s]  

10-rillington-place https://image.tmdb.org/t/p/original/aQP2rSIciaeb2FGPeOQIWsCysts.jpg
10-things-i-hate-about-you https://image.tmdb.org/t/p/original/ujERk3aKABXU3NDXOAxEQYTHe9A.jpg


...

  0%|          | 5/12137 [00:01<53:41,  3.77it/s]

10-years https://image.tmdb.org/t/p/original/yJFUBE5VGjQaTLuyhf4Hb926mn4.jpg


  0%|          | 6/12137 [00:01<56:27,  3.58it/s]

10000-bc https://image.tmdb.org/t/p/original/9I7gV6wRbGnbfI3XOKjHeLMjYEo.jpg


KeyboardInterrupt: 